# 09. 대규모 데이터 처리를 위한 희소행렬
- 현실에서 다루어야 하는 데이터는 방대한 데이터
    - 예) `MovieLens 20M`데이터는 138,494명의 사용자가 131,263개의 영화에 대해서 평가한 약 2,000만 개의 데이터를 포함
    
    $\rightarrow$ 이런 데이터를 처리할 때 가장 흔하게 직면하는 문제는 메모리의 한계

    - 이전에는 Full Matrix로 변환 (Pivot Wider)하여 처리하였지만, 방대한 데이터에서는 불가
    
- **추천 시스템에서 사용되는 대부분의 데이터는 Full Matrix로 변환하면 많은 원소가 비어있는 (결측값, 평가되지 않음) 희소행렬 (Sparse Matrix)**

    - 예) `MovieLens 100K` 데이터를 Full Matrix로 변환하면 원소 중 값(평점)을 갖는 비율이 약 0.6%에 불과


<br>

## 09.01. Sparse Matrix의 사용
- 희소행렬을 다룬다면, 전체 행렬을 저장하는 대신, 원소 중 실제로 값을 갖는 것 만 그 인덱스와 같이 저장하는 것이 훨씬 효율적
- **아래와 같이 Full Matrix를 Sparse Matrix로 처리하면, 원소의 인덱스와 그 값에 대한 정보만 저장**

    $\rightarrow$ **데이터가 희박할수록 (실제 데이터가 있는 원소의 비율이 적을수록) 저장공간이 절약되는 정도가 증가**
    
<img src ='https://www.researchgate.net/publication/328995968/figure/fig4/AS:693582436528129@1542374347304/Illustration-of-the-sparse-matrix-format-A-Example-matrix-of-size-8-8-with-5.png' width="800">

<br>

#### 희소행렬의 단점
- **데이터를 저장하거나 읽어올 때마다 값이 존재하는지 확인하여, 그에 맞는 처리가 필요**
    
    $\rightarrow$ **데이터 처리의 overhead cost가 높음**
    
    $\rightarrow$ **데이터가 희소하지 않은 경우에는 장점보다 단점이 더 부각됨**
    

<br>

### 희소행렬 구현
- Python에서의 희소행렬 종류
1. `csc_matrix` : Compressed Sparse Column Format - 효율적인 컬럼 슬라이싱
2. `csr_matric` : Compressed SParse Rows Format - 효율적인 로우 슬라이싱
3. `bsr_matrix` : Block Sparse Rows Format
4. `lil_matrix` : List of Lists Format
5. `dok_matrix` : Dictionary of Keys Format
6. `coo_matrix` : COOrdinate Format (AKA. IJV, Triplet Format)
7. `dia_matrix` : DIAgonal Format

<br>

- `csr_matrix` 

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

- 임의의 DF

In [9]:
ratings = {
    'user_id': [1,2,4], 
    'movie_id': [2,3,7], 
    'rating': [4,3,1]
}

ratings = pd.DataFrame(ratings)
ratings

,user_id,movie_id,rating
0,1,2,4
1,2,3,3
2,4,7,1


- `pandas`를 이용한 Full Matrix

In [10]:
rating_matrix = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
full_matrix1 = np.array(rating_matrix)
full_matrix1

array([[4., 0., 0.],
       [0., 3., 0.],
       [0., 0., 1.]])

<br>

- Full Matrix를 희소행렬로 변환 & 희소행렬 연산

In [19]:
data = np.array(ratings['rating']) # 값 저장
row_indices = np.array(ratings['user_id']) # 행 인덱스 저장
col_indices = np.array(ratings['movie_id']) # 열 인덱스 저장

In [30]:
rating_matrix = csr_matrix((data, (row_indices, col_indices)), dtype=int)
print(rating_matrix)

  (1, 2)	4
  (2, 3)	3
  (4, 7)	1


In [29]:
print(rating_matrix * 2)

  (1, 2)	8
  (2, 3)	6
  (4, 7)	2


In [26]:
print(rating_matrix.T)

  (2, 1)	4
  (3, 2)	3
  (7, 4)	1


In [27]:
print(rating_matrix.dot(rating_matrix.T))

  (1, 1)	16
  (2, 2)	9
  (4, 4)	1


<br>

- 희소행렬을 Full Matrix로 변환

In [22]:
full_matrix2 = rating_matrix.toarray()
full_matrix2

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 0, 0],
       [0, 0, 0, 3, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

<br>

## 09.02. 희소행렬의 추천 알고리즘 적용

In [43]:
import numpy as np
import pandas as pd

In [44]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ratings-20m.csv', names=r_cols,  sep=',',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int) 

- **인덱스와, 값을 가지는 희소행렬**

In [45]:
print(ratings.shape)
ratings.head()

(20000263, 3)


,user_id,movie_id,rating
0,1,2,3
1,1,29,3
2,1,32,3
3,1,47,3
4,1,50,3


In [46]:
from sklearn.utils import shuffle

In [47]:
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=1)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

- 희소행렬 객체로 저장

In [48]:
from scipy.sparse import csr_matrix

In [49]:
data = np.array(ratings['rating'])
row_indices = np.array(ratings['user_id'])
col_indices = np.array(ratings['movie_id'])
ratings = csr_matrix((data, (row_indices, col_indices)), dtype=int)

- MF 엔진 구현

In [51]:
class NEW_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):
        self.R = ratings # 희소행렬 객체를 그대로 사용
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.verbose = verbose

    
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        
        return np.sqrt(np.mean(self.errors**2))

    
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    
    def set_test(self, ratings_test):
        
        test_set = []
        for i in range(len(ratings_test)):              # 테스트 데이터에 있는 각 데이터에 대해서
            x, y, z = ratings_test.iloc[i]              # 인덱스 (사용자 ID, 아이템ID), 값 (평점) 가져오기
            test_set.append([x, y, z])
            self.R[x, y] = 0                       
            
        self.test_set = test_set
        
        return test_set               
    
    def test_rmse(self):
        
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
            
        return np.sqrt(error/len(self.test_set))

    def test(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]

        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i+1, rmse1, rmse2))
            if self.verbose:
#                 if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
                    
        return training_process

    def get_one_prediction(self, user_id, item_id):
        return self.get_prediction(user_id, item_id)

In [ ]:
R_temp = ratings.copy()
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=10, verbose=True)
test_set = mf.set_test(ratings_test)
result = mf.test()

Iteration: 1 ; Train RMSE = 0.9058 ; Test RMSE = 0.9648
Iteration: 2 ; Train RMSE = 0.8867 ; Test RMSE = 0.9448
Iteration: 3 ; Train RMSE = 0.8763 ; Test RMSE = 0.9341
Iteration: 4 ; Train RMSE = 0.8695 ; Test RMSE = 0.9273
